In [1]:
import pandas as pd
import pymongo
import numpy as np

In [2]:
client = pymongo.MongoClient()
db = client.swatford

In [5]:
ctd = pd.read_table("../data/CTD/CTD_chem_gene_ixns.tsv",
              comment="#",header=None,names=["chem_name",
                                             "chem_id",
                                             "casrn",
                                             "gene_symbol",
                                             "gene_id",
                                             "gene_forms","organism","tax_id","ixn","ixn_axn","pmid"])

In [8]:
ctd = ctd[["gene_id","tax_id","pmid"]].copy()

In [18]:
data = {"gene_id":[],"tax_id":[],"pmid":[]}

In [19]:
for row in ctd.itertuples():
    pmids = [int(x) for x in row.pmid.split("|")]
    data["gene_id"] += [int(row.gene_id)]*len(pmids)
    data["tax_id"] += [int(row.tax_id)]*len(pmids) if not np.isnan(row.tax_id) else [None]*len(pmids)
    data["pmid"] += pmids

In [25]:
to_insert = []
for i,row in pd.DataFrame(data).drop_duplicates().iterrows():
    r = dict(row)
    for k,v in dict(row).items():
        if np.isnan(v):
            del r[k]
    to_insert.append(r)
    if i%25000==0:
        db.ctd.insert_many(to_insert)
        to_insert=[]
db.ctd.insert_many(to_insert)